In [ ]:
# Importing Libraries
import gymnasium as gym
import torch

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [ ]:
# Environment - Lunar Lander V2
env = gym.make("LunarLander-v2")
observation, info = env.reset()

print("Observation Space Shape: ", env.observation_space.shape)
print("Action Space Shape: ",env.action_space.n)

<h1>Understand the Environment</h1>

<h2>Action Space</h2>
<p>There are four discrete actions available:</p>
<ul class="simple">
<li><p>0: do nothing</p></li>
<li><p>1: fire left orientation engine</p></li>
<li><p>2: fire main engine</p></li>
<li><p>3: fire right orientation engine</p></li>
</ul>

<h2>Observation Space</h2>
<p>There are eight different variables available:</p>
<ul>
<li><p>Horizontal pad coordinate (x)</p></li>
<li>Vertical pad coordinate (y)</p></li>
<li><p>Horizontal speed (x)</p></li>
<li><p>Vertical speed (y)</p></li>
<li><p>Angle</p></li>
<li>Vertical pad coordinate (y)</p></li>
<li><p>If the left leg contact point has touched the land (boolean)</p></li>
<li><p>If the right leg contact point has touched the land (boolean)</p></li>
</ul>

<h2>Rewards</h2>
<p>After every step a reward is granted. The total reward of an episode is the sum of the rewards for all the steps within that episode.</p>
<p>For each step, the reward:</p>
<ul>
<li><p>is increased/decreased the closer/further the lander is to the landing pad</p></li>
<li><p>is increased/decreased the slower/faster the lander is moving.</p></li>
<li><p>is decreased the more the lander is tilted (angle not horizontal)</p></li>
<li><p>is increased by 10 points for each leg that is in contact with the ground</p></li>
<li><p>is decreased by 0.03 points each frame a side engine is firing.</p></li>
<li><p>is decreased by 0.3 points each frame the main engine is firing</p></li>
</ul>

<p>The episode receive an additional reward of -100 or +100 points for crashing or landing safely respectively.</p>
<p>An episode is considered a solution if it scores at least 200 points. </p>

In [ ]:
# Create a vectorized environment - a method for stacking multiple independent environments into a single environment
env = make_vec_env("LunarLander-v2", n_envs=16)

In [ ]:
# model
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    learning_rate=0.001,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 5,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1,
    vf_coef= 0.5, 
    max_grad_norm= 0.5,
    device='mps')

In [ ]:
# Train the PPO Agent
model.learn(total_timesteps=2500000) # 2.5 million timesteps

# save the model
model_name = 'PPO-LunarLander-v2'
model.save(model_name)

In [ ]:
# Evaluate the model
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=50, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")